In [1]:
import time
import re
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

# Configuração do WebDriver
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)



# Missão

In [10]:

try:
    # Exemplo usando CSS Selector na classe QvGrid que costuma ser o container
    scrollable_element = driver.find_element(By.CSS_SELECTOR, ".QvGrid .qv-grid-content-scroller")
except:
    # Fallback: Usando o seletor específico que você mandou (ajuste se necessário)
    # Note que para seletores CSS que começam com número (#9...), precisa escapar.
    # Vamos tentar pegar pelo XPATH que é mais seguro nesse caso de IDs dinâmicos
    scrollable_element = driver.find_element(By.XPATH, "//*[@class='QvGrid']//div[contains(@style, 'overflow')]")

#

In [49]:
with open('html.txt', 'w') as f:
    f.write(driver.page_source)

In [43]:
from bs4 import BeautifulSoup
import re
import pandas as pd
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin

# --- CONFIGURAÇÃO ---
seletor_container = ".QvGrid"
urls_unicas = set()
tentativas_sem_novos = 0

# Localiza o container uma vez só
container = driver.find_element(By.CSS_SELECTOR, seletor_container)
scroll_origin = ScrollOrigin.from_element(container)

print("Iniciando scraping com ScrollOrigin...")

# Loop de Extração
for i in range(500): # Limite de segurança aumentado
    
    # 1. Extração (BeautifulSoup)
    # Pegamos o HTML atual para ler o que está na tela
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    links = soup.find_all('a', href=re.compile(r'datSessao='))
    
    qtd_antes = len(urls_unicas)
    
    for link in links:
        urls_unicas.add(link.get('href'))
        
    novos = len(urls_unicas) - qtd_antes
    print(f"Iteração {i}: {len(urls_unicas)} sessões acumuladas (+{novos} novas)")

    # 2. Critério de Parada
    if novos == 0:
        tentativas_sem_novos += 1
        # Se rodar 15 vezes sem achar nada novo, assume que acabou
        if tentativas_sem_novos >= 15:
            print("Parece que chegamos ao fim da lista.")
            break
    else:
        tentativas_sem_novos = 0

    # 3. O SEU SCROLL (Wheel Input)
    # Aumentei o delta_y para 150 para ser um pouco mais rápido, 
    # mas mantendo a suavidade para o site carregar.
    # Se começar a pular datas, diminua para 50 ou 10.
    
    try:
        ActionChains(driver).scroll_from_origin(
            scroll_origin=scroll_origin, 
            delta_x=0, 
            delta_y=8
        ).perform()
    except Exception as e:
        print(f"Erro no scroll: {e}")
        # Tenta recuperar o elemento caso o DOM tenha mudado (stale element)
        container = driver.find_element(By.CSS_SELECTOR, seletor_container)
        scroll_origin = ScrollOrigin.from_element(container)

    # Pequena pausa para o 'virtual DOM' renderizar as novas linhas
    time.sleep(0.5)

# --- PROCESSAMENTO FINAL ---
print(f"\nFinalizado! Total de {len(urls_unicas)} sessões únicas coletadas.")

dados = []
for href in urls_unicas:
    match = re.search(r'datSessao=(\d{2}/\d{2}/\d{4})', href)
    if match:
        dados.append({'data': match.group(1), 'url': href})

df_premiacoes = pd.DataFrame(dados)
if not df.empty:
    df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')
    df = df.sort_values('data', ascending=False)
    print(df.head())
    # df.to_csv('sessoes_senado_final.csv', index=False)
else:
    print("Nenhum dado encontrado.")

Iniciando scraping com ScrollOrigin...
Iteração 0: 7 sessões acumuladas (+7 novas)
Iteração 1: 7 sessões acumuladas (+0 novas)
Iteração 2: 7 sessões acumuladas (+0 novas)
Iteração 3: 7 sessões acumuladas (+0 novas)
Iteração 4: 7 sessões acumuladas (+0 novas)
Iteração 5: 7 sessões acumuladas (+0 novas)
Iteração 6: 7 sessões acumuladas (+0 novas)
Iteração 7: 7 sessões acumuladas (+0 novas)
Iteração 8: 7 sessões acumuladas (+0 novas)
Iteração 9: 7 sessões acumuladas (+0 novas)
Iteração 10: 7 sessões acumuladas (+0 novas)
Iteração 11: 7 sessões acumuladas (+0 novas)
Iteração 12: 7 sessões acumuladas (+0 novas)
Iteração 13: 7 sessões acumuladas (+0 novas)
Iteração 14: 7 sessões acumuladas (+0 novas)
Iteração 15: 7 sessões acumuladas (+0 novas)
Parece que chegamos ao fim da lista.

Finalizado! Total de 7 sessões únicas coletadas.
          data                                                url
85  2025-12-19  http://legis.senado.leg.br/diarios/BuscaDiario...
133 2025-12-17  http://legis.sen

In [44]:
dfs = [df_premiacoes,df_tematicos,df_especiais,df_n_deliberativo,df_deliberativo]

df_final = pd.concat(dfs)

In [48]:
df_deliberativo.to_csv('Datas_presenças_deliberativas.csv')

In [ ]:


# --- 3. PROCESSAMENTO FINAL ---

dados_finais = []
for href in urls_encontradas:
    # Extrair a data com Regex
    match = re.search(r'datSessao=(\d{2}/\d{2}/\d{4})', href)
    if match:
        dados_finais.append({
            'data': match.group(1),
            'url': href
        })

df = pd.DataFrame(dados_finais)

# Ordenar por data (opcional)
df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')
df = df.sort_values('data', ascending=False)

print(df)

In [52]:
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Base URL para completar o link (pois o href vem como "/diarios/ver/...")
BASE_URL = "https://legis.senado.leg.br"

def extrair_link_do_calendario(driver, data_obj):
    """
    Recebe um objeto datetime, ajusta o calendário e retorna o link do diário.
    """
    # Formata para o input do site
    mes_ano = data_obj.strftime("%m/%Y")
    dia = str(data_obj.day)
    
    try:
        # 1. Mudar o Mês (Input)
        campo_mes = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.ID, "input_mes"))
        )
        
        # Verifica se já está no mês certo para ganhar tempo (opcional, mas ajuda)
        valor_atual = campo_mes.get_attribute("value")
        if valor_atual != mes_ano:
            campo_mes.click()
            # Limpa o campo (Ctrl+A + Delete é mais seguro que .clear() em inputs mascarados)
            campo_mes.send_keys(Keys.CONTROL + "a")
            campo_mes.send_keys(Keys.DELETE)
            campo_mes.send_keys(mes_ano)
            campo_mes.send_keys(Keys.ENTER)
            time.sleep(1.5) # Espera o AJAX carregar os dias

        # 2. Encontrar o elemento do dia e pegar o HREF
        # Procura um link (a) que tenha o número do dia exato
        xpath_dia = f"//div[@id='calendario_container']//a[normalize-space()='{dia}']"
        
        elemento_dia = driver.find_element(By.XPATH, xpath_dia)
        link_relativo = elemento_dia.get_attribute("href")
        
        # Reconstrói o link completo
        if link_relativo:
            return f"{BASE_URL}{link_relativo}" if link_relativo.startswith("/") else link_relativo
            
    except Exception as e:
        # Se der erro (ex: dia não tem link, ou calendário falhou), retorna None
        # print(f"Não achou link para {data_obj}: {e}")
        return None
    
    return None

# --- APLICAÇÃO NO SEU DATAFRAME ---

print("Iniciando coleta dos links de PDF...")

# Cria uma lista para armazenar os resultados
links_encontrados = []

# Garante que a coluna 'data' é datetime
df_deliberativo['data'] = pd.to_datetime(df_deliberativo['data'])

# Itera sobre cada linha do DataFrame
for index, row in df_deliberativo.iterrows():
    data_atual = row['data']
    
    print(f"Buscando: {data_atual.strftime('%d/%m/%Y')} ...")
    
    link = extrair_link_do_calendario(driver, data_atual)
    
    links_encontrados.append(link)
    
    if link:
        print(f" -> Link: {link}")
    else:
        print(" -> Nenhum link encontrado.")

# Adiciona a nova coluna ao DataFrame original
df_deliberativo['link_pdf'] = links_encontrados

# Salva o resultado
df_deliberativo.to_csv("resultado_com_links_pdf.csv", index=False)
print("\nConcluído! DataFrame atualizado:")
print(df_deliberativo.head())

Iniciando coleta dos links de PDF...
Buscando: 23/09/2025 ...


C:\Users\rafae_9y1k772\AppData\Local\Temp\ipykernel_30184\1776124271.py:61: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_deliberativo['data'] = pd.to_datetime(df_deliberativo['data'])


 -> Link: https://legis.senado.leg.br/diarios/ver/122097
Buscando: 22/10/2025 ...
 -> Link: https://legis.senado.leg.br/diarios/ver/122538
Buscando: 25/03/2025 ...
 -> Link: https://legis.senado.leg.br/diarios/ver/119796
Buscando: 08/07/2025 ...
 -> Link: https://legis.senado.leg.br/diarios/ver/121136
Buscando: 02/12/2025 ...
 -> Link: https://legis.senado.leg.br/diarios/ver/123097
Buscando: 25/11/2025 ...
 -> Link: https://legis.senado.leg.br/diarios/ver/122917
Buscando: 24/06/2025 ...
 -> Link: https://legis.senado.leg.br/diarios/ver/120836
Buscando: 11/11/2025 ...
 -> Link: https://legis.senado.leg.br/diarios/ver/122817
Buscando: 12/11/2025 ...
 -> Link: https://legis.senado.leg.br/diarios/ver/122697
Buscando: 30/09/2025 ...
 -> Link: https://legis.senado.leg.br/diarios/ver/122317
Buscando: 20/05/2025 ...
 -> Link: https://legis.senado.leg.br/diarios/ver/120436
Buscando: 29/10/2025 ...
 -> Link: https://legis.senado.leg.br/diarios/ver/122637
Buscando: 24/04/2025 ...
 -> Link: https:

In [50]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def selecionar_data_calendario(driver, mes_ano, dia):
    """
    Seleciona uma data no calendário do Senado.
    mes_ano: string formato "MM/AAAA" (ex: "11/2025")
    dia: string ou int (ex: 15)
    """
    print(f"--- Tentando selecionar {dia} de {mes_ano} ---")

    # 1. ENCONTRAR E PREENCHER O MÊS
    # O ID foi identificado no seu arquivo html.txt linha 87
    campo_mes = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "input_mes"))
    )
    
    # Limpa o campo (às vezes .clear() falha em inputs com máscara, então usamos backspace)
    campo_mes.click()
    campo_mes.send_keys(Keys.CONTROL + "a")
    campo_mes.send_keys(Keys.DELETE)
    
    # Digita o novo mês e aperta ENTER para disparar a atualização do calendário
    campo_mes.send_keys(mes_ano)
    campo_mes.send_keys(Keys.ENTER)
    
    # Pausa essencial: O site demora um pouco para recarregar a tabelinha dos dias via AJAX
    time.sleep(2) 

    # 2. ENCONTRAR E CLICAR NO DIA
    # Lógica: Procurar dentro do #calendario_container uma célula (td) ou link (a)
    # que tenha exatamente o texto do número do dia, ignorando espaços.
    
    dia = str(dia) # Garante que é string
    
    # XPATH explicado:
    # //div[@id='calendario_container'] -> Garante que estamos olhando só para o calendário
    # //a[normalize-space()='{dia}']   -> Procura um LINK que o texto limpo seja igual ao dia
    # Usamos 'a' (link) porque no HTML os dias clicáveis são links (href).
    xpath_dia = f"//div[@id='calendario_container']//a[normalize-space()='{dia}']"
    
    try:
        botao_dia = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, xpath_dia))
        )
        botao_dia.click()
        print("Dia selecionado com sucesso!")
        return True
    except:
        print(f"O dia {dia} não parece estar clicável ou disponível neste mês.")
        return False

# --- COMO USAR NO SEU SCRIPT ---
# Supondo que você já abriu o modal "Abrir um diário"
# Selecionar Novembro de 2025, dia 12
selecionar_data_calendario(driver, "11/2025", 12)

--- Tentando selecionar 12 de 11/2025 ---
Dia selecionado com sucesso!


True

In [ ]:
driver.get('https://www25.senado.leg.br/web/senadores/em-exercicio/-/e/por-nome')

In [71]:
senadores = driver.find_elements(By.CSS_SELECTOR, '#senadoresemexercicio-tabela-senadores tr > td > a')

lista = []

for s in senadores:
    if (s.text.lower().strip() == 'esperidião amin') or (s.text.lower().strip() == f'jorge seif') or (s.text.lower().strip() == f'ivete da silveira'):
        registro = {
            'nome':s.text,
            'link':s.get_attribute('href')
        }
        lista.append(registro)

In [72]:
lista

[{'nome': 'Esperidião Amin',
  'link': 'https://www25.senado.leg.br/web/senadores/senador/-/perfil/22'},
 {'nome': 'Ivete da Silveira',
  'link': 'https://www25.senado.leg.br/web/senadores/senador/-/perfil/6010'},
 {'nome': 'Jorge Seif',
  'link': 'https://www25.senado.leg.br/web/senadores/senador/-/perfil/6342'}]

In [ ]:
dfs = []
for l in lista:
    driver.get(l['link'])
    html = driver.page_source
    time.sleep(2)
    try:
        
        df_exterior = pd.read_html(html, attrs={'title':'Lista de Missões do Senador'})[0]
        df_exterior['senador'] = l['nome']
        df_exterior['tipo'] = 'Exterior'
        dfs.append(df_exterior)
    except:
        print(l['nome'], 'nao tem missoes no exterior')
        
    try:
        df_brasil = pd.read_html(html, attrs={'title':'Lista de Missões do Senador'})[1]
        df_brasil['senador'] = l['nome']
        df_brasil['tipo'] = 'Brasil'
        dfs.append(df_brasil)
    except:
        print(l['nome'], 'nao tem missoes no brasil')

In [111]:
df_final_missoes = pd.concat(dfs)

In [113]:
import pandas as pd

# 1. Faz o split (divisão) da string usando o separador " a "
# O expand=True cria um novo DataFrame com duas colunas temporárias
datas_separadas = df_final_missoes['Período'].str.split(' a ', expand=True)

# 2. Atribui às novas colunas
df_final_missoes['data_inicio'] = datas_separadas[0]
df_final_missoes['data_fim'] = datas_separadas[1]

# 3. TRUQUE PARA DATAS ÚNICAS:
# Se a 'data_fim' estiver vazia (None/NaT), significa que o evento foi de 1 dia só.
# Então preenchemos o 'fim' com a mesma data do 'inicio'.
df_final_missoes['data_fim'] = df_final_missoes['data_fim'].fillna(df_final_missoes['data_inicio'])

# 4. Converter para formato de data real (datetime) para poder ordenar/filtrar depois
df_final_missoes['data_inicio'] = pd.to_datetime(df_final_missoes['data_inicio'], format='%d/%m/%Y', errors='coerce')
df_final_missoes['data_fim'] = pd.to_datetime(df_final_missoes['data_fim'], format='%d/%m/%Y', errors='coerce')

# (Opcional) Remover a coluna antiga 'Período' se não precisar mais
# df_final_missoes = df_final_missoes.drop(columns=['Período'])

# Conferindo o resultado
print(df_final_missoes[['Período', 'data_inicio', 'data_fim']].head(10))

                   Período data_inicio   data_fim
0  31/05/1996 a 13/06/1996  1996-05-31 1996-06-13
1  24/02/2019 a 27/02/2019  2019-02-24 2019-02-27
2  11/04/2019 a 12/04/2019  2019-04-11 2019-04-12
3  16/09/2019 a 23/09/2019  2019-09-16 2019-09-23
4  25/11/2019 a 29/11/2019  2019-11-25 2019-11-29
5  23/02/2020 a 26/02/2020  2020-02-23 2020-02-26
6  28/02/2022 a 03/03/2022  2022-02-28 2022-03-03
7  26/02/2023 a 06/03/2023  2023-02-26 2023-03-06
8               11/12/2023  2023-12-11 2023-12-11
9  25/02/2024 a 28/02/2024  2024-02-25 2024-02-28


In [115]:
# Filtra apenas as linhas onde o ano de início é 2025
df_missoes_2025 = df_final_missoes[df_final_missoes['data_inicio'].dt.year == 2025].copy()

# (Opcional) Ordenar por data para ficar organizado
df_missoes_2025 = df_missoes_2025.sort_values('data_inicio')

# Exibe o resultado
print(f"Total de missões em 2025: {len(df_missoes_2025)}")
print(df_missoes_2025[['Evento', 'data_inicio', 'data_fim']].head())

# Salva apenas as de 2025
# df_missoes_2025.to_csv("missoes_2025.csv", index=False)

Total de missões em 2025: 14
                                               Evento data_inicio   data_fim
5   Conservative Political Action Conference - CPA...  2025-02-19 2025-02-22
14                         Mobile World Congress 2025  2025-03-02 2025-03-05
15        Visita Técnica aos Parlamentares Britânicos  2025-03-17 2025-03-19
16  Sistemas de Saúde da Associação Nacional de Ho...  2025-04-13 2025-04-16
6                   2ª Cúpula SulAmericana AgroGlobal  2025-04-23 2025-04-24


In [117]:
df_sheets = pd.read_csv('./Links presenças - Página1.csv')

In [119]:
import pandas as pd

# --- ETAPA 1: Garantir que tudo é Data (Datetime) ---
# Isso é crucial para a comparação funcionar (maior que / menor que)
df_sheets['data'] = pd.to_datetime(df_sheets['data'])

# Certifique-se de usar o DataFrame de missões já tratado no passo anterior (com data_inicio e data_fim)
# Vou chamar de 'df_missoes' aqui, mas use o nome da sua variável (ex: df_final_missoes ou df_missoes_2025)
df_missoes = df_final_missoes.copy() # Usando uma cópia para segurança
df_missoes['data_inicio'] = pd.to_datetime(df_missoes['data_inicio'])
df_missoes['data_fim'] = pd.to_datetime(df_missoes['data_fim'])

# --- ETAPA 2: Identificar os Senadores ---
lista_senadores = df_missoes['senador'].dropna().unique()
print(f"Senadores encontrados: {lista_senadores}")

# --- ETAPA 3: Função de Verificação ---
def verificar_status(data_sessao, nome_senador, df_viagens):
    """
    Verifica se na 'data_sessao' o 'nome_senador' estava viajando.
    """
    # 1. Filtra apenas as viagens desse senador específico
    viagens_senador = df_viagens[df_viagens['senador'] == nome_senador]
    
    # 2. Verifica se a data da sessão está DENTRO de algum intervalo de viagem
    # Lógica: (Inicio da Viagem <= Sessão) E (Fim da Viagem >= Sessão)
    conflito = ((viagens_senador['data_inicio'] <= data_sessao) & 
                (viagens_senador['data_fim'] >= data_sessao)).any()
    
    if conflito:
        return "Em missão"
    else:
        return "Disponível"

# --- ETAPA 4: Criar as Colunas no df_sheets ---
print("\nCruzando dados...")

for senador in lista_senadores:
    # Cria uma nova coluna com o nome do senador
    # E aplica a função linha por linha
    df_sheets[senador] = df_sheets['data'].apply(
        lambda d: verificar_status(d, senador, df_missoes)
    )

print("Concluído!")

# --- ETAPA 5: Visualizar e Salvar ---
# Mostra as primeiras linhas com as novas colunas
print(df_sheets.head())

# Salva o resultado final
df_sheets.to_csv("sessoes_com_presenca_calculada.csv", index=False)

Senadores encontrados: ['Esperidião Amin' 'Jorge Seif']

Cruzando dados...
Concluído!
        data                                                url  \
0 2025-09-23  http://legis.senado.leg.br/diarios/BuscaDiario...   
1 2025-10-22  http://legis.senado.leg.br/diarios/BuscaDiario...   
2 2025-03-25  http://legis.senado.leg.br/diarios/BuscaDiario...   
3 2025-07-08  http://legis.senado.leg.br/diarios/BuscaDiario...   
4 2025-12-02  http://legis.senado.leg.br/diarios/BuscaDiario...   

                                         link_pdf Status  Nome arquivo  \
0  https://legis.senado.leg.br/diarios/ver/122097  Feito  SESSÃO_23_09   
1  https://legis.senado.leg.br/diarios/ver/122538  Feito  SESSÃO_22_10   
2  https://legis.senado.leg.br/diarios/ver/119796  Feito  SESSÃO_25_03   
3  https://legis.senado.leg.br/diarios/ver/121136  Feito  SESSÃO_08_07   
4  https://legis.senado.leg.br/diarios/ver/123097  Feito  SESSÃO_02_12   

  Esperidião Amin  Jorge Seif  
0      Disponível  Disponível  
1 

In [ ]:
import os
from pypdf import PdfReader
import pandas as pd

# 1. Definir os Senadores que vamos monitorar (deve ser igual ao nome na coluna e no PDF)
# DICA: Use o nome exato que aparece no PDF.
senadores_alvo = ["Esperidião Amin", "Jorge Seif"]

# 2. Função para ler o PDF e buscar o nome
def verificar_presenca_pdf(nome_arquivo_base, nome_senador):
    """
    Tenta abrir o PDF (com ou sem til) e procura o nome do senador.
    Retorna: True (Presente), False (Ausente), ou None (Arquivo não achado)
    """
    # Tenta nome sem acento (SESSAO_...)
    caminho_1 = f"{nome_arquivo_base}.pdf"
    # Tenta nome com acento (SESSÃO_...) - Substitui apenas o primeiro 'AO'
    caminho_2 = f"{nome_arquivo_base.replace('SESSAO', 'SESSÃO')}.pdf"
    
    arquivo_final = None
    
    if os.path.exists(caminho_1):
        arquivo_final = caminho_1
    elif os.path.exists(caminho_2):
        arquivo_final = caminho_2
        
    if not arquivo_final:
        return None # Não encontrou o arquivo PDF

    try:
        reader = PdfReader(arquivo_final)
        texto_completo = ""
        for pagina in reader.pages:
            texto_completo += pagina.extract_text() + "\n"
        
        # Busca insensível a maiúsculas/minúsculas
        if nome_senador.upper() in texto_completo.upper():
            return True
        else:
            return False
    except Exception as e:
        print(f"Erro ao ler {arquivo_final}: {e}")
        return None

# 3. Aplicar a lógica no DataFrame
print("Iniciando verificação nos PDFs...")

for index, row in df_sheets.iterrows():
    nome_arquivo = row['Nome arquivo'] # Ex: SESSAO_23_09
    
    for senador in senadores_alvo:
        status_anterior = row[senador] # Pega o status que calculamos antes (Em missão/Disponível)
        
        # Verifica no PDF
        resultado_pdf = verificar_presenca_pdf(nome_arquivo, senador)
        
        novo_status = status_anterior
        
        if resultado_pdf is True:
            novo_status = "Presente"
        elif resultado_pdf is False:
            # Se não está no PDF, mantemos 'Em missão' se já estava, ou virar 'Ausente'
            if status_anterior == "Em missão":
                novo_status = "Em missão" # Mantém a justificativa
            else:
                novo_status = "Ausente"
        elif resultado_pdf is None:
            # Se não achou o PDF, avisa (opcional)
            novo_status = f"{status_anterior} (PDF não achado)"

        # Atualiza a célula
        df_sheets.at[index, senador] = novo_status

print("\nVerificação concluída!")
print(df_sheets[['data', 'Nome arquivo'] + senadores_alvo].head(10))

# Salvar
df_sheets.to_csv("presenca_final_consolidada.csv", index=False)

,data,url,link_pdf,Status,Nome arquivo,Esperidião Amin,Jorge Seif
0,2025-09-23,http://legis.senado.leg.br/diarios/BuscaDiario...,https://legis.senado.leg.br/diarios/ver/122097,Feito,SESSÃO_23_09,Disponível,Disponível
1,2025-10-22,http://legis.senado.leg.br/diarios/BuscaDiario...,https://legis.senado.leg.br/diarios/ver/122538,Feito,SESSÃO_22_10,Disponível,Disponível
2,2025-03-25,http://legis.senado.leg.br/diarios/BuscaDiario...,https://legis.senado.leg.br/diarios/ver/119796,Feito,SESSÃO_25_03,Disponível,Disponível
3,2025-07-08,http://legis.senado.leg.br/diarios/BuscaDiario...,https://legis.senado.leg.br/diarios/ver/121136,Feito,SESSÃO_08_07,Disponível,Disponível
4,2025-12-02,http://legis.senado.leg.br/diarios/BuscaDiario...,https://legis.senado.leg.br/diarios/ver/123097,Feito,SESSÃO_02_12,Disponível,Disponível
...,...,...,...,...,...,...,...
71,2025-09-16,http://legis.senado.leg.br/diarios/BuscaDiario...,https://legis.senado.leg.br/diarios/ver/121977,Feito,SESSAO_16_09,Disponível,Disponível
72,2025-04-30,http://legis.senado.leg.br/diarios/BuscaDiario...,https://legis.senado.leg.br/diarios/ver/120177,Feito,SESSAO_30_04,Em missão,Em missão
73,2025-03-19,http://legis.senado.leg.br/diarios/BuscaDiario...,https://legis.senado.leg.br/diarios/ver/119736,Feito,SESSAO_19_03,Em missão,Disponível
74,2025-07-01,http://legis.senado.leg.br/diarios/BuscaDiario...,https://legis.senado.leg.br/diarios/ver/120936,Feito,SESSAO_01_07,Disponível,Disponível


In [132]:
import os
from pypdf import PdfReader
import pandas as pd

# CONFIGURAÇÃO
PASTA_PDFS = "Arquivos pdfs" 

# LISTA ATUALIZADA (Adicionei a Ivete aqui manualmente)
senadores_alvo = ["Esperidião Amin", "Jorge Seif", "Ivete da Silveira"]

def verificar_presenca_pdf(nome_arquivo_base, nome_senador):
    """
    Procura o PDF dentro da pasta 'Arquivos pdfs', tenta com e sem til,
    e verifica se o nome do senador consta no texto.
    """
    # Monta os caminhos possíveis
    caminho_1 = os.path.join(PASTA_PDFS, f"{nome_arquivo_base}.pdf")
    nome_com_til = nome_arquivo_base.replace('SESSAO', 'SESSÃO')
    caminho_2 = os.path.join(PASTA_PDFS, f"{nome_com_til}.pdf")
    
    arquivo_final = None
    
    if os.path.exists(caminho_1):
        arquivo_final = caminho_1
    elif os.path.exists(caminho_2):
        arquivo_final = caminho_2
        
    if not arquivo_final:
        return None 

    try:
        reader = PdfReader(arquivo_final)
        texto_completo = ""
        for pagina in reader.pages:
            texto_completo += pagina.extract_text() + "\n"
        
        # Verifica se o nome está lá (Case Insensitive)
        if nome_senador.upper() in texto_completo.upper():
            return True 
        else:
            return False 
            
    except Exception as e:
        print(f"Erro ao ler {arquivo_final}: {e}")
        return None

# --- LOOP PRINCIPAL ---
print(f"Iniciando verificação para: {senadores_alvo}")

for index, row in df_sheets.iterrows():
    nome_arquivo = row['Nome arquivo']
    
    for senador in senadores_alvo:
        # TRUQUE PARA A IVETE (e outros sem missão):
        # Se a coluna do senador não existir no DataFrame (porque não veio das missões),
        # criamos ela agora e assumimos que o status inicial é "Disponível".
        if senador not in df_sheets.columns:
            status_anterior = "Disponível"
        else:
            # Se já existe, pega o valor (pode ser "Em missão" ou "Disponível")
            status_anterior = row[senador]
            # Se estiver vazio ou nulo, considera Disponível
            if pd.isna(status_anterior) or status_anterior == "":
                status_anterior = "Disponível"
        
        # Verifica no PDF
        resultado_pdf = verificar_presenca_pdf(nome_arquivo, senador)
        
        novo_status = status_anterior
        
        if resultado_pdf is True:
            novo_status = "Presente"
            
        elif resultado_pdf is False:
            # Se não está no PDF:
            if status_anterior == "Em missão":
                novo_status = "Em missão" # Mantém a justificativa
            else:
                novo_status = "Ausente" # Se estava Disponível e não foi, é falta
                
        elif resultado_pdf is None:
            novo_status = "PDF não encontrado"

        # Grava no DataFrame (cria a coluna se não existir)
        df_sheets.at[index, senador] = novo_status

print("\n--- Processamento Concluído! ---")
print(df_sheets[['data', 'Nome arquivo'] + senadores_alvo].head(10))

# Salva
df_sheets.to_csv("presenca_final_senadores.csv", index=False)

Iniciando verificação para: ['Esperidião Amin', 'Jorge Seif', 'Ivete da Silveira']

--- Processamento Concluído! ---
        data  Nome arquivo Esperidião Amin Jorge Seif Ivete da Silveira
0 2025-09-23  SESSÃO_23_09        Presente   Presente          Presente
1 2025-10-22  SESSÃO_22_10        Presente   Presente          Presente
2 2025-03-25  SESSÃO_25_03        Presente   Presente          Presente
3 2025-07-08  SESSÃO_08_07        Presente    Ausente          Presente
4 2025-12-02  SESSÃO_02_12        Presente   Presente          Presente
5 2025-11-25  SESSÃO_25_11        Presente   Presente          Presente
6 2025-06-24  SESSÃO_24_06        Presente  Em missão          Presente
7 2025-11-11  SESSÃO_11_11        Presente   Presente          Presente
8 2025-11-12  SESSÃO_12_11        Presente   Presente          Presente
9 2025-09-30  SESSÃO_30_09        Presente   Presente          Presente


In [133]:
df_sheets

,data,url,link_pdf,Status,Nome arquivo,Esperidião Amin,Jorge Seif,Ivete da Silveira
0,2025-09-23,http://legis.senado.leg.br/diarios/BuscaDiario...,https://legis.senado.leg.br/diarios/ver/122097,Feito,SESSÃO_23_09,Presente,Presente,Presente
1,2025-10-22,http://legis.senado.leg.br/diarios/BuscaDiario...,https://legis.senado.leg.br/diarios/ver/122538,Feito,SESSÃO_22_10,Presente,Presente,Presente
2,2025-03-25,http://legis.senado.leg.br/diarios/BuscaDiario...,https://legis.senado.leg.br/diarios/ver/119796,Feito,SESSÃO_25_03,Presente,Presente,Presente
3,2025-07-08,http://legis.senado.leg.br/diarios/BuscaDiario...,https://legis.senado.leg.br/diarios/ver/121136,Feito,SESSÃO_08_07,Presente,Ausente,Presente
4,2025-12-02,http://legis.senado.leg.br/diarios/BuscaDiario...,https://legis.senado.leg.br/diarios/ver/123097,Feito,SESSÃO_02_12,Presente,Presente,Presente
...,...,...,...,...,...,...,...,...
71,2025-09-16,http://legis.senado.leg.br/diarios/BuscaDiario...,https://legis.senado.leg.br/diarios/ver/121977,Feito,SESSAO_16_09,Presente,Presente,Presente
72,2025-04-30,http://legis.senado.leg.br/diarios/BuscaDiario...,https://legis.senado.leg.br/diarios/ver/120177,Feito,SESSAO_30_04,Em missão,Em missão,Ausente
73,2025-03-19,http://legis.senado.leg.br/diarios/BuscaDiario...,https://legis.senado.leg.br/diarios/ver/119736,Feito,SESSAO_19_03,Presente,Presente,Ausente
74,2025-07-01,http://legis.senado.leg.br/diarios/BuscaDiario...,https://legis.senado.leg.br/diarios/ver/120936,Feito,SESSAO_01_07,Presente,Presente,Presente


# Preposicao

In [2]:
import time
import re
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC

# Configuração do WebDriver

driver = webdriver.Chrome()

In [3]:
driver.implicitly_wait(10)

In [5]:
driver.get('https://www25.senado.leg.br/web/atividade/materias')

In [22]:
links_preposicoes = [
    {    'nome':'Ivete da Silveira', 'link':'https://www25.senado.leg.br/web/atividade/materias?p_p_id=materia_WAR_atividadeportlet&p_p_lifecycle=0&_materia_WAR_atividadeportlet_pesquisaAvancada=true&_materia_WAR_atividadeportlet_ano=2025&_materia_WAR_atividadeportlet_tipoAutor=SENADOR&_materia_WAR_atividadeportlet_dataApresentacaoInicial=&_materia_WAR_atividadeportlet_numero=&_materia_WAR_atividadeportlet_situacaoTramitacao=TODAS&_materia_WAR_atividadeportlet_dataApresentacaoFinal=&_materia_WAR_atividadeportlet_anoNorma=&_materia_WAR_atividadeportlet_codigoClasseTematica=&_materia_WAR_atividadeportlet_palavraChave=&_materia_WAR_atividadeportlet_tipoNorma=&_materia_WAR_atividadeportlet_codigoParlamentar=6010&_materia_WAR_atividadeportlet_numeroNorma=&_materia_WAR_atividadeportlet_nomeRelator=&_materia_WAR_atividadeportlet_btnSubmit=&_materia_WAR_atividadeportlet_nomeAutor=&_materia_WAR_atividadeportlet_codigoTipoConteudo=&_materia_WAR_atividadeportlet_siglasTipos=PL%2CPDL%2CPLP%2CDESTPEC%2CDESTPLN&_materia_WAR_atividadeportlet_p=',},
    {    'nome': 'Esperidião Amin','link': 'https://www25.senado.leg.br/web/atividade/materias?p_p_id=materia_WAR_atividadeportlet&p_p_lifecycle=0&_materia_WAR_atividadeportlet_pesquisaAvancada=true&_materia_WAR_atividadeportlet_ano=2025&_materia_WAR_atividadeportlet_tipoAutor=SENADOR&_materia_WAR_atividadeportlet_dataApresentacaoInicial=&_materia_WAR_atividadeportlet_numero=&_materia_WAR_atividadeportlet_situacaoTramitacao=TODAS&_materia_WAR_atividadeportlet_dataApresentacaoFinal=&_materia_WAR_atividadeportlet_anoNorma=&_materia_WAR_atividadeportlet_codigoClasseTematica=&_materia_WAR_atividadeportlet_palavraChave=&_materia_WAR_atividadeportlet_tipoNorma=&_materia_WAR_atividadeportlet_codigoParlamentar=22&_materia_WAR_atividadeportlet_numeroNorma=&_materia_WAR_atividadeportlet_nomeRelator=&_materia_WAR_atividadeportlet_btnSubmit=&_materia_WAR_atividadeportlet_nomeAutor=&_materia_WAR_atividadeportlet_codigoTipoConteudo=&_materia_WAR_atividadeportlet_siglasTipos=PL%2CPDL%2CPLP%2CDESTPEC%2CDESTPLN&_materia_WAR_atividadeportlet_p=',},
    {    'nome':'Jorge Seif','link': 'https://www25.senado.leg.br/web/atividade/materias?p_p_id=materia_WAR_atividadeportlet&p_p_lifecycle=0&_materia_WAR_atividadeportlet_pesquisaAvancada=true&_materia_WAR_atividadeportlet_ano=2025&_materia_WAR_atividadeportlet_tipoAutor=SENADOR&_materia_WAR_atividadeportlet_dataApresentacaoInicial=&_materia_WAR_atividadeportlet_numero=&_materia_WAR_atividadeportlet_situacaoTramitacao=TODAS&_materia_WAR_atividadeportlet_dataApresentacaoFinal=&_materia_WAR_atividadeportlet_anoNorma=&_materia_WAR_atividadeportlet_codigoClasseTematica=&_materia_WAR_atividadeportlet_palavraChave=&_materia_WAR_atividadeportlet_tipoNorma=&_materia_WAR_atividadeportlet_codigoParlamentar=6342&_materia_WAR_atividadeportlet_numeroNorma=&_materia_WAR_atividadeportlet_nomeRelator=&_materia_WAR_atividadeportlet_btnSubmit=&_materia_WAR_atividadeportlet_nomeAutor=&_materia_WAR_atividadeportlet_codigoTipoConteudo=&_materia_WAR_atividadeportlet_siglasTipos=PL%2CPDL%2CPLP%2CDESTPEC%2CDESTPLN&_materia_WAR_atividadeportlet_p='}
]


In [24]:
df_link_prepo = pd.DataFrame(links_preposicoes)

In [31]:
driver.find_element(By.LINK_TEXT, 'Próxima').get_property('parentElement').get_property('previousElementSibling').text

'5'

In [52]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def scrapy_prepo_v2(nome_sen):
    # Salva o identificador da aba principal (onde está a lista)
    janela_principal = driver.current_window_handle
    
    dados_extraidos = []

    # Encontra todos os blocos de proposições
    blocos = driver.find_elements(By.CSS_SELECTOR, "div.sf-proposicoes-resultado > div")

    for bloco in blocos:
        try:
            # Verifica estrutura básica
            if not bloco.find_elements(By.CSS_SELECTOR, "dl"):
                continue

            item = {}

            # --- 1. Extração da Listagem (Dados Básicos) ---
            
            # Elemento do Link (precisamos do objeto web element para pegar o texto e do href para abrir)
            elem_link_materia = bloco.find_element(By.CSS_SELECTOR, "dd:nth-of-type(1) .span6:first-child a")
            link_url = elem_link_materia.get_attribute("href")

            item['materia'] = elem_link_materia.text
            item['ementa'] = bloco.find_element(By.CSS_SELECTOR, "dd:nth-of-type(2)").text
            item['autor'] = bloco.find_element(By.CSS_SELECTOR, "dd:nth-of-type(3)").text
            item['data'] = bloco.find_element(By.CSS_SELECTOR, "dd:nth-of-type(4)").text

            # Tramitação
            elem_tramitacao = bloco.find_element(By.CSS_SELECTOR, "dd:nth-of-type(1) .pull-right")
            item['tramitacao_texto'] = elem_tramitacao.text
            item['tramitacao_link'] = elem_tramitacao.get_attribute("href")
            
            # Adiciona o nome do senador passado na função
            item['Senador_pesquisa'] = nome_sen

            # --- 2. Extração Detalhada (Nova Aba) ---
            
            # Abre nova aba com o link capturado
            driver.execute_script("window.open(arguments[0]);", link_url)
            
            # Muda o foco do Selenium para a nova aba (a última da lista)
            driver.switch_to.window(driver.window_handles[-1])

            try:
                # Wait Explícito: Espera até 5 segundos para a div de texto carregar
                wait = WebDriverWait(driver, 5)
                
                # A. Captura do ASSUNTO
                # Procura a div que contém os parágrafos condensados
                container_textos = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.sf-bloco-paragrafos-condensados")))
                paragrafos = container_textos.find_elements(By.TAG_NAME, "p")
                
                item['assunto'] = "Não informado" # Valor padrão
                for p in paragrafos:
                    if "Assunto:" in p.text:
                        # Remove o rótulo "Assunto:" e espaços extras
                        item['assunto'] = p.text.replace("Assunto:", "").strip()
                        break

                # B. Captura da SITUAÇÃO ATUAL
                try:
                    # Busca a label dentro da caixa azul do dashboard
                    elem_situacao = driver.find_element(By.CSS_SELECTOR, ".bg_blue-dashboard span.label")
                    item['situacao_atual'] = elem_situacao.text
                except:
                    item['situacao_atual'] = "Não identificada"

            except Exception as e_detalhes:
                print(f"Erro nos detalhes da matéria {item['materia']}: {e_detalhes}")
                item['assunto'] = "Erro na extração"
                item['situacao_atual'] = "Erro na extração"
            
            finally:
                # IMPORTANTE: Bloco finally garante que a aba fecha mesmo se der erro acima
                driver.close() # Fecha aba atual
                driver.switch_to.window(janela_principal) # Volta para a lista

            # Adiciona o item completo à lista
            dados_extraidos.append(item)

        except Exception as e:
            # Log de erro silencioso para o loop principal
            print(f"Erro genérico ao processar item: {e}")
            
            # Segurança extra: se o erro aconteceu e a aba extra ainda estiver aberta, fecha ela
            if len(driver.window_handles) > 1:
                driver.close()
                driver.switch_to.window(janela_principal)
            continue

    print(f"Total extraído para {nome_sen}: {len(dados_extraidos)}")
    return dados_extraidos

In [43]:
def scrapy_prepo(nome_sen):

    dados_extraidos = []

    # Encontra todos os blocos de proposições (cada div filho direto dentro do container principal)
    # O seletor > div garante que pegamos apenas os filhos diretos
    blocos = driver.find_elements(By.CSS_SELECTOR, "div.sf-proposicoes-resultado > div")

    for bloco in blocos:
        try:
            # Verifica se o bloco possui a estrutura de lista (dl) para evitar divs vazias ou de formatação
            if not bloco.find_elements(By.CSS_SELECTOR, "dl"):
                continue

            item = {}

            # 1. Matéria (Primeiro link dentro do primeiro dd)
            # O seletor pega o primeiro 'dd', entra na primeira coluna 'span6' e pega o 'a'
            item['materia'] = bloco.find_element(By.CSS_SELECTOR, "dd:nth-of-type(1) .span6:first-child a").text

            # 2. Ementa (Texto do segundo dd)
            item['ementa'] = bloco.find_element(By.CSS_SELECTOR, "dd:nth-of-type(2)").text

            # 3. Autor (Texto do terceiro dd)
            item['autor'] = bloco.find_element(By.CSS_SELECTOR, "dd:nth-of-type(3)").text

            # 4. Data (Texto do quarto dd)
            item['data'] = bloco.find_element(By.CSS_SELECTOR, "dd:nth-of-type(4)").text

            # 5. Tramitação e Link
            # A tramitação fica no segundo 'span6' (ou classe pull-right) dentro do primeiro 'dd'
            elem_tramitacao = bloco.find_element(By.CSS_SELECTOR, "dd:nth-of-type(1) .pull-right")
            
            item['tramitacao_texto'] = elem_tramitacao.text
            item['tramitacao_link'] = elem_tramitacao.get_attribute("href")
            
            item['Senador_pesquisa'] = nome_sen
            dados_extraidos.append(item)

        except Exception as e:
            # Log de erro silencioso para não parar o script em escala
            print(f"Erro ao extrair item: {e}")
            continue

    # Exemplo de verificação rápida
    print(f"Total extraído: {len(dados_extraidos)}")
    return dados_extraidos

In [54]:
lsita = scrapy_prepo_v2('ESPERIDIAO')

Total extraído para ESPERIDIAO: 8


In [56]:
lista_prepo = []
for index, linha in df_link_prepo.iterrows():
    link = linha['link'] + '1'
    driver.get(link)
    
    time.sleep(2)
    
    pagina_limite = driver.find_element(By.LINK_TEXT, 'Próxima').get_property('parentElement').get_property('previousElementSibling').text
    print('Pagina limite de', linha['nome'], '=',pagina_limite)
    
    pagina_limite = int(pagina_limite)
    
    for i in range(1,pagina_limite+1):
        print(f'Acessando página {i} de {pagina_limite}')
        
        link = linha['link'] + f'{i}'
        driver.get(link)
        
        registro = scrapy_prepo_v2(nome_sen=linha['nome'])
        lista_prepo.extend(registro)

Pagina limite de Ivete da Silveira = 2
Acessando página 1 de 2
Total extraído para Ivete da Silveira: 10
Acessando página 2 de 2
Total extraído para Ivete da Silveira: 2
Pagina limite de Esperidião Amin = 3
Acessando página 1 de 3
Total extraído para Esperidião Amin: 10
Acessando página 2 de 3
Total extraído para Esperidião Amin: 10
Acessando página 3 de 3
Total extraído para Esperidião Amin: 8
Pagina limite de Jorge Seif = 5
Acessando página 1 de 5
Total extraído para Jorge Seif: 10
Acessando página 2 de 5
Total extraído para Jorge Seif: 10
Acessando página 3 de 5
Total extraído para Jorge Seif: 10
Acessando página 4 de 5
Total extraído para Jorge Seif: 10
Acessando página 5 de 5
Total extraído para Jorge Seif: 1


In [57]:
lista_prepo

[{'materia': 'PLP 60/2025',
  'ementa': 'Altera a Lei Complementar nº 123, de 14 de dezembro de 2006, para modificar os valores de enquadramento do Microempreendedor Individual (MEI) e permitir a contratação de até dois empregados.',
  'autor': 'Senadora Ivete da Silveira (MDB/SC)',
  'data': '11/03/2025',
  'tramitacao_texto': 'Tramitação Bicameral do(a) PLP 60/2025',
  'tramitacao_link': 'https://www.congressonacional.leg.br/materias/materias-bicamerais/-/ver/plp-60-2025',
  'Senador_pesquisa': 'Ivete da Silveira',
  'assunto': 'Economia e Desenvolvimento > Indústria, Comércio e Serviços > Micro e Pequenas Empresas',
  'situacao_atual': 'Em tramitação'},
 {'materia': 'PL 3545/2025',
  'ementa': 'Altera a Lei nº 14.541, de 3 de abril de 2023, para dispor que as Delegacias Especializadas de Atendimento à Mulher serão chefiadas, preferencialmente, por delegada de polícia civil.',
  'autor': 'Senadora Ivete da Silveira (MDB/SC)',
  'data': '17/07/2025',
  'tramitacao_texto': 'Tramitação 

In [59]:
pd.DataFrame(lista_prepo).to_csv('Preposicoes.csv', index=False)